Необхідні бібліотеки

In [1]:
import cv2
import numpy as np
import os

In [2]:
import time

In [3]:
start_time = time.time()
end_time = time.time()

execution_time = end_time - start_time
print("Час виконання:", execution_time, "секунд")

Час виконання: 0.0009999275207519531 секунд


Переробити зображення в масив

In [4]:
# перероблюємо зображення в масив чб
def convert_of_images(image_path):
    img = cv2.imread(image_path, 0) // 255
    return img

In [5]:
# Функція активації ReLU
def relu(x):
    return np.maximum(0, x)

Функція згортки

In [6]:
def convolution(image, kernel, stride=1, padding=0):
    # додаємо padding
    image = np.pad(image, [(padding, padding), (padding, padding)], mode='constant', constant_values=0)

    kernel_height, kernel_width = kernel.shape
    padded_height, padded_width = image.shape

    # Визначаємо розміри вихідного зображення
    output_height = (padded_height - kernel_height) // stride + 1
    output_width = (padded_width - kernel_width) // stride + 1

    # вихідне зображення, .astype(np.float32) - для скорочення(прискорює, без 0,(3), з 0,3334)
    new_image = np.zeros((output_height, output_width)).astype(np.float32)

    # множимо зображення на фільтр
    for y in range(0, output_height):
        for x in range(0, output_width):
            # елем. зображення*елем.фільтра, потім сумуємо і ділимо на кількість елем. в фільтрі(9, оскільки всі фільтри розміром 3x3)
            region = image[y * stride:y * stride + kernel_height, x * stride:x * stride + kernel_width]
            sum_of_products = np.sum(region * kernel)/9
            new_image[y][x] = relu(sum_of_products).astype(np.float32)

    return new_image

def conv(img, filters):
    conv = []
    for i in img:
        for j in filters:
            new = convolution(i, j)
            conv.append(new)

    return conv

Пулінг

In [7]:
def max_pooling(image, pool_size = 2):
    height, width = image.shape
    pooled_height = height // pool_size
    pooled_width = width // pool_size
    pooled_image = np.zeros((pooled_height, pooled_width))

    for i in range(pooled_height):
        for j in range(pooled_width):
            start_row = i * pool_size
            end_row = start_row + pool_size
            start_col = j * pool_size
            end_col = start_col + pool_size
            # вирізаємо елементи, якi були на координатах... знаходимо макс і записуємо в новий масив
            pooled_image[i, j] = np.max(image[start_row:end_row, start_col:end_col])

    return pooled_image

def pool(img):
    pool = []
    for i in img:
        new = max_pooling(i)
        pool.append(new)

    return pool

Карта ознак(згортка кожного каналу)

In [8]:
def sign_channel(img, len_filters):
    dict = []

    for j in range(len_filters):
        res = np.zeros((len(img[0]), len(img[0])), dtype=int)
        for i in range(0, len(img), len_filters):
            res = res + img[i+j]
        dict.append(res)

    return dict

Алгоритм згортки

In [9]:
def feature_extraction(image_path):
    img = [convert_of_images(image_path)]

    filter1 = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]])
    filter2 = np.array([[0, 0, 0], [1, 1, 1], [0, 0, 0]])
    filter3 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    filter4 = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
    filter5 = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])

    filter6 = np.array([[-1, -1, 0], [-1,  0, 1], [ 0,  1, 1]])
    filter7 = np.array([[1, 1, 0], [1,  0, -1], [ 0,  -1, -1]])
    filter8 = np.array([[-1, -1, -1], [-1,  8, -1], [-1, -1, -1]])
    filter9 = np.array([[-1, -1, -1], [-1,  9, -1], [-1, -1, -1]])
    filter10 = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    filter11 = np.array([[-1,  0, 1], [-1,  0, 1], [-1,  0, 1]])
    filter12 = np.array([[1,  0, -1], [1,  0, -1], [1,  0, -1]])
    
    filter13 = np.array([[-1, -1, -1], [2,  2, 2], [-1, -1, -1]])
    filter14 = np.array([[-1, 0, 1], [-2,  0, 2], [-1, 0, 1]])
    filter15 = np.array([[-1, -2, -1], [0,  0, 0], [1, 2, 1]])
    filter16 = np.array([[1, 2, 1], [2,  4, 2], [1, 2, 1]])
    filter17 = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    filter18 = np.array([[-1, -1, -1], [-1,  8, -1], [-1, -1, -1]])
    filter19 = np.array([[1,  -2, -1], [-2,  4, -2], [1,  -2, 1]])
    filter20 = np.array([[1,  1, 1], [1,  -2, 1], [-1,  -1, -1]])
    filter21 = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    
    filters_1 = [filter1, filter2, filter3, filter4, filter5]
    filters_2 = [filter6, filter7, filter8, filter9, filter10, filter11, filter12]
    filters_3 = [filter13, filter14, filter15, filter16, filter17, filter18, filter19, filter20, filter21]
    
    img_1 = conv(img, filters_1)
    img_2 = pool(img_1)

    img_3 = conv(img_2, filters_2)
    img_4 = sign_channel(img_3, len(filters_2))
    img_5 = pool(img_4)
    
    img_6 = conv(img_5, filters_3)
    img_7 = sign_channel(img_6, len(filters_3))
    img_8 = pool(img_7)

    # print(img_8)
    # перебираємо шари, flatten - перероблює в одномірний масив(ну всі рядки пишуться в 1[])
    res = np.concatenate([i.flatten() for i in img_8])

    # print(res)
    # print(len(res))
    
    return res#.reshape(-1, 1)

Дані

In [15]:
# сума міток, має = 1
start_time = time.time()

data = []

for root, dirs, files in os.walk("Фігури/Circle"):
    for file in files:
        # print(f"Фігури/Circle/{file}")
        data.append((feature_extraction(f"Фігури/Circle/{file}"), 0))
        
print(len(data))

for root, dirs, files in os.walk("Фігури/Square"):
    for file in files:
        data.append((feature_extraction(f"Фігури/Square/{file}"), 0.3))
        
print(len(data))

for root, dirs, files in os.walk("Фігури/Triangle"):
    for file in files:
        data.append((feature_extraction(f"Фігури/Triangle/{file}"), 0.7))

# print(data[0])
print(len(data))
end_time = time.time()

execution_time = end_time - start_time
print("Час виконання:", execution_time, "секунд")


# data.append((feature_extraction("Фігури/Circle/circle_1.png"), 0))
# data.append((feature_extraction("Фігури/Square/square_1.png"), 0.3))
# data.append((feature_extraction("Фігури/Triangle/triangle_1.png"), 0.7))

# print(len(data))
# for input_stats, correct_predict in data:
#     print(input_stats.shape)
#     print(f"correct_predict: {correct_predict}")

160
320
480
Час виконання: 514.9179999828339 секунд


In [11]:
start_time = time.time()

#ТЕСТОВІ ДАННІ
data_test = []

for root, dirs, files in os.walk("Фігури/Test"):
    for file in files:
        data_test.append((feature_extraction(f"Фігури/Test/{file}"), f"{file}"))

end_time = time.time()

execution_time = end_time - start_time
# print(len(data_test))
print("Час виконання:", execution_time, "секунд")

Час виконання: 117.9539999961853 секунд


In [23]:
#рандомні коефіцієнти, які змінюються при навчанні
weights_0_1 = np.random.normal(0.0, 1, (len(data[0][0])//2,len(data[0][0])))
weights_1_2 = np.random.normal(0.0, 1, (1,len(data[0][0])//2))
# print(len(weights_0_1))
# print(len(weights_0_1[0]))

Класифікація(функції)

In [27]:
# Якщо є більше двох класів викор. функцію softmax

def sigmoida(x):
    return 1 / (1 + np.exp(-x))

func_sigmoida = np.vectorize(sigmoida)

# функція помилки
def MSE(y, Y):
    return np.mean((y - Y) ** 2)

def predict(inputs):
    inputs_1 = np.dot(weights_0_1, inputs)
    outputs_1 = func_sigmoida(inputs_1)
    # print(outputs_1)
    # print(outputs_1.reshape(1, len(outputs_1)))

    inputs_2 = np.dot(weights_1_2, outputs_1)
    outputs_2 = func_sigmoida(inputs_2)

    return outputs_2

def train(inputs, expected_predict):
    
    inputs_1 = np.dot(weights_0_1, inputs)
    outputs_1 = func_sigmoida(inputs_1)

    inputs_2 = np.dot(weights_1_2, outputs_1)
    outputs_2 = func_sigmoida(inputs_2)

    # список з 1 числа
    # результат, який ми отримали
    actual_predict = outputs_2[0]

    # print(f"actual_predict: {actual_predict}")
    # print(weights_1_2)
    # print(weights_1_2.shape) #розміри (255,)
    
    # помилки останнього шару
    error_layer_2 = np.array([actual_predict - expected_predict])
    # print(f"error_layer_2: {error_layer_2}")
    gradient_layer_2 = actual_predict * (1 - actual_predict)
    weights_delta_layer_2 = error_layer_2 * gradient_layer_2
    # print(gradient_layer_2)
    # print(weights_delta_layer_2)
    # Оновлення ваг для вихідного шару
    calc_weights_1_2 = weights_1_2 - (np.dot(weights_delta_layer_2, outputs_1.reshape(1, len(outputs_1))) * learning_rate)

    error_layer_1 = weights_delta_layer_2 * weights_1_2
    gradient_layer_1 = outputs_1 * (1 - outputs_1)
    weights_delta_layer_1 = error_layer_1 * gradient_layer_1
    # print(len(weights_delta_layer_1))

    calc_weights_0_1 = weights_0_1 - (np.dot(inputs.reshape(len(inputs), 1), weights_delta_layer_1).T * learning_rate)

    return calc_weights_0_1, calc_weights_1_2

Тренування моделі

In [28]:
start_time = time.time()
#проес тренування (перші 2 числа обираємо експерементально)
epochs = 10
learning_rate = 0.05
t_l = []

for e in range(epochs):
    inputs_end = []
    correct_predictions = []

    for input_stats, correct_predict in data:
        # print(len(input_stats))
        weights_0_1, weights_1_2 = train(input_stats, correct_predict)
        inputs_end.append(np.array(input_stats))
        correct_predictions.append(np.array(correct_predict))

    train_loss = MSE(predict(np.array(inputs_end).T), np.array(correct_predictions))
    t_l.append(train_loss)
print("кінець навчання")
end_time = time.time()

execution_time = end_time - start_time
print("Час виконання:", execution_time, "секунд")
t_l

кінець навчання
Час виконання: 4.292999982833862 секунд


[1.0725918390149453e-05,
 1.0713553168765945e-05,
 1.070121971100243e-05,
 1.0688917880361861e-05,
 1.0676647541171065e-05,
 1.0664408558577001e-05,
 1.0652200798556075e-05,
 1.0640024127893227e-05,
 1.0627878414176706e-05,
 1.0615763525797218e-05]

Прогнозування та виведення результатів

In [29]:
start_time = time.time()

for input_stat, correct_predict in data:
    prediction = predict(np.array(input_stat))
  
    print(f"Правильна відповідь: {correct_predict}, передбачення мережі: {prediction}") 
end_time = time.time()

execution_time = end_time - start_time
print("Час виконання:", execution_time, "секунд")

Правильна відповідь: 0, передбачення мережі: [0.00222103]
Правильна відповідь: 0, передбачення мережі: [0.00123503]
Правильна відповідь: 0, передбачення мережі: [0.00514804]
Правильна відповідь: 0, передбачення мережі: [0.00400277]
Правильна відповідь: 0, передбачення мережі: [0.00415717]
Правильна відповідь: 0, передбачення мережі: [0.00599833]
Правильна відповідь: 0, передбачення мережі: [0.00466961]
Правильна відповідь: 0, передбачення мережі: [0.00466961]
Правильна відповідь: 0, передбачення мережі: [0.00415717]
Правильна відповідь: 0, передбачення мережі: [0.00197792]
Правильна відповідь: 0, передбачення мережі: [0.00211332]
Правильна відповідь: 0, передбачення мережі: [0.00415717]
Правильна відповідь: 0, передбачення мережі: [0.00211332]
Правильна відповідь: 0, передбачення мережі: [0.00365683]
Правильна відповідь: 0, передбачення мережі: [0.00599833]
Правильна відповідь: 0, передбачення мережі: [0.00509415]
Правильна відповідь: 0, передбачення мережі: [0.00509415]
Правильна відп

In [31]:
start_time = time.time()

for input_stat, correct_predict in data_test:
    prediction = predict(np.array(input_stat))
  
    print(f"Правильна відповідь: {correct_predict}, передбачення мережі: {prediction}") 
end_time = time.time()

execution_time = end_time - start_time
print("Час виконання:", execution_time, "секунд")

Правильна відповідь: circle_161.png, передбачення мережі: [0.00599833]
Правильна відповідь: circle_162.png, передбачення мережі: [0.00123503]
Правильна відповідь: circle_163.png, передбачення мережі: [0.00466961]
Правильна відповідь: circle_164.png, передбачення мережі: [0.00739292]
Правильна відповідь: circle_165.png, передбачення мережі: [0.00509415]
Правильна відповідь: circle_166.png, передбачення мережі: [0.00365683]
Правильна відповідь: circle_167.png, передбачення мережі: [0.00211332]
Правильна відповідь: circle_168.png, передбачення мережі: [0.00365683]
Правильна відповідь: circle_169.png, передбачення мережі: [0.00599833]
Правильна відповідь: circle_170.png, передбачення мережі: [0.00062741]
Правильна відповідь: circle_171.png, передбачення мережі: [0.00197792]
Правильна відповідь: circle_172.png, передбачення мережі: [0.00365683]
Правильна відповідь: circle_173.png, передбачення мережі: [0.0088792]
Правильна відповідь: circle_174.png, передбачення мережі: [0.00211332]
Правиль

In [ ]:
print(weights_0_1) #коефіціянти які ми шукали
print(weights_1_2)